In [4]:
import time, tweepy, sys
import pandas as pd
sys.path.insert(1, '../../../scripts/')
from s3_support import *

In [2]:
CONSUMER_KEY = "aqB8jd0zgfMs4N9XNn6BA"
CONSUMER_SECRET = "LsitOJMTvsksgYHlwlW3PQSPlMBh2WyS0vmS1PnGs"

In [6]:
forms = redshift_query_read("select id, org, type, path from form where status=1", schema="production")
print(len(forms))
forms.tail(3)

23655


,id,org,type,path
23652,966308,436247,1,5fhivh
23653,966324,430140,1,e2c
23654,966340,443121,1,qupwbd


In [35]:
seconds_delay = max(float(len(forms)) / float(100000), 1.0)
seconds_delay = 5.0
counter = 0

print("TWITTER SPIDER: crawling {} forms with a {} second delay between requests".format(len(forms), seconds_delay))

TWITTER SPIDER: crawling 23655 forms with a 5.0 second delay between requests


In [11]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth)

In [41]:
data = []
for _, form in forms.iterrows():
    # build form URL to search for
    form_url = "url:secure.qgiv.com/for/{}".format(form['path'])
    
    if form_url is not None and form_url != "":
        try:
            for e in tweepy.Cursor(api.search, q=form_url).items():
                data.append({
                    'tweet_id': e.id,
                    'twitter_user_id': e.user.id,
                    'form': form['id'],
                    'org': form['org'],
                    'url': form_url.replace('url:', ''),
                    'date_created': e.created_at,
                    'content': e.text,
                    'retweet_count': e.retweet_count,
                    'like_count': e.favorite_count,
                    'hashtags': ";".join([t['text'] for t in e.entities['hashtags']])
                })
        except Exception as e:
            # possible too fast, slow it down
            if seconds_delay < 1.0:
                print("increasing delay after exception ({})".format(e))
                seconds_delay += 1.0
            else:
                print("unaccounted for exception: {}".format(e))
                break
    time.sleep(seconds_delay)
    if len(data) > 100:
        break

unaccounted for exception: sequence item 0: expected str instance, dict found


In [44]:
data[:2]

[{'tweet_id': 1333546403814109186,
  'twitter_user_id': 393125026,
  'form': 848,
  'org': 859,
  'url': 'secure.qgiv.com/for/faithfulfathering',
  'date_created': datetime.datetime(2020, 11, 30, 23, 0, 13),
  'content': 'Every dollar donated empowers us to help dads become the heroes their families need. \nDonate now:… https://t.co/H7DkH2b7Ko',
  'retweet_count': 0,
  'like_count': 0,
  'hashtags': ''},
 {'tweet_id': 1331667221840617473,
  'twitter_user_id': 21540170,
  'form': 42040,
  'org': 14261,
  'url': 'secure.qgiv.com/for/aaogc',
  'date_created': datetime.datetime(2020, 11, 25, 18, 33, 1),
  'content': 'In March, ArtsBuild immediately set up an emergency relief fund and provided checks to more than 100 individual art… https://t.co/0FYFTdKaKv',
  'retweet_count': 0,
  'like_count': 4,
  'hashtags': ''}]